In [1]:
#Importing the required modules:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
import gensim
import gensim.models.keyedvectors as word2vec

In [2]:
#Loading the modles:
fname = 'C:/Users/Suryansh/OneDrive - University College Dublin/Desktop/ucd/Stage4/FYP/New/Models/bias'

In [3]:
model = gensim.models.keyedvectors.KeyedVectors.load(fname)

In [4]:
max_size = len(model.wv.vocab)-1
w2v = np.zeros((max_size,model.layer1_size))
w2v

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Seedlist to associate topics with

In [5]:
african_list = ['african', 'africans', 'blacks', 'kaffir', 'kaffer', 'cafri', 'mulatto', 'mulattoes', 'negress', 'negro', 'negroes', 'nigger', 'pygmy', 'pygmies', 'slave', 'slaves', 'creole', 'creoles', 'native', 'natives']

In [6]:
cauc_list = ['britsh','britishers','english','englishman','englishmen','european','europeans', 'americans', 'yankees', 'french', 'portugese', 'dutch']

### Dictionaries representing concepts of interest:

In [7]:
unciv_concept = ['uncivilised','uncivilized','wild','uncultured','barbaric','barbarous','barbarian','boorish','churlish','disrespectful','impolite','rude','unconscionable','vulgar','brutish','coarse','crass','crude','discourteous','gross','impertinent','loutish','mannerless','outrageous','philistine','primitive','rugged','savage','uncontrolled','uncouth','uncultivated','uneducated','ungodly','unholy','unmannered','unpolished','unrefined','unsophisticated','wicked','untamed','beastily','beastial','immoral','senseless','mindless']

In [8]:
civ_concept = ['civilised','civilized','civil','cultured','educated','enlightened','humane','refined','sophisticated','advanced','urbane','cultivated','polite','genteel','courteous','mannerly','respectful','developed','complaisant','advanced','elegant','polished','gracious','domesticated','industrial']

In [9]:
### Continuing the tensorboard code:

In [10]:
# Assigning labels to dictionary terms I'm interested in analysing
with open("metadata.tsv","w+") as file_metadata:
    file_metadata.write("Word\tCategory\n")
    for i,word in enumerate(model.wv.index2word[:max_size]):
        w2v[i] = model.wv[word]
        if(word in unciv_concept):
            file_metadata.write("%s\t%s\n" % (word,'terms_uncivilised'))
        elif(word in civ_concept):
            file_metadata.write("%s\t%s\n" % (word,'terms_civilised'))
        elif(word in african_list):
            file_metadata.write("%s\t%s\n" % (word,'terms_africans'))
        elif(word in cauc_list):
            file_metadata.write("%s\t%s\n" % (word,'terms_caucasian'))
        else:
            file_metadata.write("%s\t%s\n" % (word,'none'))

In [11]:
w2v

array([[-2.03846812e+00,  2.50827193e+00,  4.16977048e-01, ...,
        -3.06430817e+00, -1.03770852e+00,  1.49504697e+00],
       [-8.28388214e-01,  5.27804613e-01,  3.30885410e+00, ...,
        -1.12653464e-01,  3.92413974e+00,  1.20077990e-01],
       [ 3.60598505e-01,  1.36050177e+00, -2.08310574e-01, ...,
        -3.76490206e-01, -5.32922924e-01,  1.65610766e+00],
       ...,
       [-2.48545338e-03,  3.17125395e-03, -3.72075569e-03, ...,
         3.82435531e-03, -7.62992480e-04,  4.36963513e-03],
       [-4.64878278e-03,  3.92424222e-03,  4.34570434e-03, ...,
         2.11275392e-03,  2.60445382e-03,  1.95318297e-03],
       [-3.21174995e-03, -2.71514873e-03,  4.67777951e-03, ...,
        -1.40409265e-03,  7.27262450e-05,  5.06083306e-04]])

In [12]:
sess = tf.InteractiveSession()

In [13]:
#Let us create a 2D tensor called embedding that holds our embeddings.
with tf.device("/cpu:0"):
    embedding = tf.Variable(w2v, trainable=False, name='embedding')

In [14]:
tf.global_variables_initializer().run()

In [15]:
path = 'tensorboard'

In [16]:
# let us create an object to Saver class which is actually used to 
#save and restore variables to and from our checkpoints
saver = tf.train.Saver()

In [17]:
# using file writer, we can save our summaries and events to our event file.
writer = tf.summary.FileWriter(path, sess.graph)

In [18]:
# adding into projector
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = 'embedding'
embed.metadata_path = 'metadata.tsv'

In [19]:
# Specify the width and height of a single thumbnail.
projector.visualize_embeddings(writer, config)

saver.save(sess, path+'/model.ckpt', global_step=max_size)

'tensorboard/model.ckpt-788791'